# Extract Procedures and Medications
These will be one table each (whether or not a patient received a procedure / medication)

In [1]:
import pandas_gbq
from google.oauth2 import service_account
import pandas as pd

In [2]:
# apply credentials
credentials = service_account.Credentials.from_service_account_file('../Patient-Similarity-credentials.json')
pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = "patient-similarity"

In [4]:
# since I have to fix some of the data issues in python, I need to get frequencies in python
def extractFrequencies(df):
    # first group the newly cleaned data
    df = df.groupby(['subject_id','label', 'value']).sum().reset_index()
    
    # now compute frequencies
    total_counts = df.groupby(['subject_id','label']).sum().reset_index()
    total_counts.columns = ['subject_id', 'label', 'total_counts']
    df = pd.merge(df, total_counts, on = ['subject_id', 'label'], how = 'left')
    df['frequency'] = df.num_occurences/df.total_counts
    return df

## Procedures

In [8]:
q = """SELECT distinct A.subject_id, B.label, 1 as value, 1 as frequency
FROM `patient-similarity.mimic.procedureevents_mv`  as A
left join  `patient-similarity.mimic.d_items` as B
on a.itemid=b.itemid
where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts`)
and A.HADM_ID is not null 
order by subject_id ,   B.LABEL """
procedures = pandas_gbq.read_gbq(q)
procedures.head()

,subject_id,label,value,frequency
0,188,16 Gauge,1,1
1,188,18 Gauge,1,1
2,188,20 Gauge,1,1
3,188,22 Gauge,1,1
4,188,Abdominal X-Ray,1,1


In [9]:
procedures.to_csv("../data/patientData/med&procs_categorical_procedures.csv", index = False)

## Medications
Leaving medications out for now. There are just way too many and it is way too messy